# Configuração do Ambiente XAdapt-Drift

Este notebook demonstra como usar a biblioteca XAdapt-Drift para análise de drift em modelos de Machine Learning.

## Configuração do PYTHONPATH

Primeiro, vamos adicionar o diretório raiz da biblioteca ao PYTHONPATH para permitir imports diretos.

In [18]:
# Configurar PYTHONPATH para importar a biblioteca XAdapt-Drift
import sys
import os
from pathlib import Path

# Obter o diretório raiz do projeto (um nível acima do diretório 'examples')
project_root = Path.cwd().parent
print(f"Diretório do projeto: {project_root}")

# Adicionar ao PYTHONPATH se ainda não estiver
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Adicionado {project_root} ao PYTHONPATH")
else:
    print("✅ Diretório do projeto já está no PYTHONPATH")

# Verificar se a biblioteca pode ser importada
try:
    import xadapt_drift
    print("✅ XAdapt-Drift importado com sucesso!")
    print(f"Localização da biblioteca: {xadapt_drift.__file__}")
except ImportError as e:
    print(f"❌ Erro ao importar XAdapt-Drift: {e}")
    print("Verifique se você está executando o notebook do diretório correto.")

Diretório do projeto: /home/alexandre/Documents/XDrift-Analyzer
✅ Diretório do projeto já está no PYTHONPATH
✅ XAdapt-Drift importado com sucesso!
Localização da biblioteca: /home/alexandre/Documents/XDrift-Analyzer/xadapt_drift/__init__.py


## 🚀 Exemplo Básico de Uso

Agora que a biblioteca está configurada, vamos demonstrar um exemplo básico de uso com o padrão adapter que discutimos anteriormente:

In [ ]:
# Exemplo prático: Usando o SklearnAdapter com a BaseAdapter
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Importar classes da nossa biblioteca
from xadapt_drift.adapters.sklearn_adapter import SklearnAdapter
from xadapt_drift.drift.detector import DriftDetector

print("🎯 Demonstração do Valor da BaseAdapter\n")

# 1. Criar dados de exemplo
print("📊 Criando dados de exemplo...")
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
feature_names = [f"feature_{i}" for i in range(10)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Treinar modelo sklearn
print("🤖 Treinando modelo RandomForest...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Criar adapter usando nossa BaseAdapter
print("🔌 Criando SklearnAdapter...")
adapter = SklearnAdapter(
    model=model,
    feature_names=feature_names,
    validate_model=True
)

print(f"✅ Adapter criado: {adapter}")
print(f"📋 Informações do modelo: {adapter.get_model_info()['model_type']}")
print(f"📊 Features: {len(adapter.feature_names)} features")

# 4. Usar o adapter (mesma interface independente do framework!)
print("\n🔮 Fazendo predições...")
predictions = adapter.predict(X_test)
print(f"✅ Predições realizadas: {predictions.shape}")

# 5. Gerar explicações
print("\n🧠 Gerando explicações SHAP...")
try:
    explanations = adapter.explain(X_test[:10], method="shap")  # Usar apenas 10 amostras
    print(f"✅ Explicações geradas para {len(explanations)} features")
    
    # Mostrar top 3 features mais importantes
    sorted_features = sorted(explanations.items(), key=lambda x: x[1], reverse=True)
    print("🏆 Top 3 features mais importantes:")
    for i, (feature, importance) in enumerate(sorted_features[:3], 1):
        print(f"   {i}. {feature}: {importance:.4f}")
        
except Exception as e:
    print(f"⚠️ SHAP não disponível ou erro: {e}")
    print("💡 Tentando método de permutation importance...")
    
    try:
        explanations = adapter.explain(X_test[:10], y_test[:10], method="permutation")
        print(f"✅ Permutation importance gerada para {len(explanations)} features")
    except Exception as e2:
        print(f"❌ Erro também com permutation: {e2}")

print("\n🎉 Exemplo concluído! A BaseAdapter forneceu:")
print("   ✓ Interface consistente independente do framework")
print("   ✓ Validação automática de entrada")
print("   ✓ Logging integrado")
print("   ✓ Tratamento de erros robusto")
print("   ✓ Metadados do modelo padronizados")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score
import sys


plt.style.use('seaborn-v0_8-pastel')
sns.set_palette('pastel')


from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Import XAdapt-Drift components
from xadapt_drift import XAdaptDrift
from xadapt_drift.adapters.sklearn_adapter import SklearnAdapter
from xadapt_drift.utils.advanced_metrics import AdvancedDriftDetector


### Criação de dados - geração de dados sintéticos e funções de indução de Drift 

In [7]:
def create_synthetic_data(n_samples=10000, n_cat_features=3, n_num_features=5, seed=42):
    """Create a synthetic dataset with mixed data types"""
    np.random.seed(seed)

    # Create numerical features
    X_numerical = np.random.randn(n_samples, n_num_features)
    
    # Create categorical features (3 categories each)
    X_categorical = np.random.randint(0, 3, size=(n_samples, n_cat_features))

    # Create target based on both numerical and categorical features
    y = (0.5 * np.sum(X_numerical[:, :2], axis=1) + 
         0.8 * (X_categorical[:, 0] == 2).astype(int) - 
         0.5 * (X_categorical[:, 1] == 0).astype(int) + 
         0.1 * np.random.randn(n_samples)) > 0
    
    # Combine features
    X = np.hstack([X_numerical, X_categorical])

    # Create feature names
    numerical_cols = [f'num_{i}' for i in range(n_num_features)]
    categorical_cols = [f'cat_{i}' for i in range(n_cat_features)]
    feature_names = numerical_cols + categorical_cols

    # Convert to DataFrame
    df = pd.DataFrame(X, columns=feature_names)
    
    # Convert categorical columns to correct type
    for col in categorical_cols:
        df[col] = df[col].astype('category')

    return df, y.astype(int), numerical_cols, categorical_cols

In [8]:
def induce_drift(df, num_cols, cat_cols, drift_type='mean_shift'):
    """Induce different types of drift in the dataset.
    Args:
        df: Original DataFrame
        num_cols: List of numerical feature names
        cat_cols: List of categorical feature names
        drift_type: Type of drift to induce ('mean_shift', 'variance_change', 'category_frequency', 'multiple')
    Returns:
        drifted_df: DataFrame with induced drift
        drifted_features: List of features that were changed
    """
    
    drifted_df = df.copy()
    
    if drift_type == 'mean_shift':
        # Shift the mean of the first numerical feature
        feature = num_cols[0]
        shift = 1.5 * drifted_df[feature].std()
        drifted_df[feature] += shift
        drifted_features = [feature]
        
    elif drift_type == 'variance_change':
        # Increase the variance of the second numerical feature
        feature = num_cols[1]
        drifted_df[feature] = drifted_df[feature] * 2.0
        drifted_features = [feature]
    
    elif drift_type == 'category_frequency':
        # Change the distribution of a categorical feature
        feature = cat_cols[0]
        # Find the least common category
        least_common = drifted_df[feature].value_counts().idxmin()
        # Make it more common by replacing some values
        mask = np.random.choice([True, False], size=len(drifted_df), p=[0.4, 0.6])
        drifted_df.loc[mask, feature] = least_common
        drifted_features = [feature]
        
    elif drift_type == 'multiple':
        # Induce multiple drifts
        # Shift mean of first numerical feature
        drifted_df[num_cols[0]] += 1.2 * drifted_df[num_cols[0]].std()
        # Increase variance of second numerical feature
        drifted_df[num_cols[1]] = drifted_df[num_cols[1]] * 1.8
        # Change categorical distribution
        feature = cat_cols[0]
        mask = np.random.choice([True, False], size=len(drifted_df), p=[0.3, 0.7])
        drifted_df.loc[mask, feature] = drifted_df[feature].value_counts().idxmin()
        drifted_features = [num_cols[0], num_cols[1], cat_cols[0]]
    
    return drifted_df, drifted_features

In [6]:
def induce_specific_drifts(test_df, feature_names):
    """Induce specific types of drift for testing different metrics."""
    
    scenarios = {}
    
    # Scenario 1: Gradual mean shift (detectable by KL/JS divergence)
    scenario_1 = test_df.copy()
    numerical_features = [f for f in feature_names if f.startswith('feature_')]
    target_feature = numerical_features[0]
    
    # Gradual shift that creates different distribution shapes
    shift_values = np.linspace(0, 2, len(scenario_1))
    scenario_1[target_feature] += shift_values * scenario_1[target_feature].std()
    scenarios['gradual_mean_shift'] = scenario_1
    
    # Scenario 2: Distribution shape change (strong KL divergence signal)
    scenario_2 = test_df.copy()
    target_feature = numerical_features[1]
    
    # Transform from normal to exponential-like distribution
    original_data = scenario_2[target_feature]
    # Apply exponential transformation while preserving some original characteristics
    transformed_data = np.random.exponential(scale=np.abs(original_data.mean()), size=len(original_data))
    scenario_2[target_feature] = transformed_data
    scenarios['distribution_shape_change'] = scenario_2
    
    # Scenario 3: Categorical frequency drift (detectable by Chi-square and categorical KL)
    scenario_3 = test_df.copy()
    
    # Change category distribution significantly
    new_categories = np.random.choice(['Type_A', 'Type_B', 'Type_C'], 
                                    size=len(scenario_3), 
                                    p=[0.1, 0.2, 0.7])  # Very different from original [0.5, 0.3, 0.2]
    scenario_3['category_1'] = new_categories
    scenarios['categorical_frequency_drift'] = scenario_3
    
    # Scenario 4: Multiple subtle drifts (low individual signals, but cumulative effect)
    scenario_4 = test_df.copy()
    
    # Small shifts in multiple features
    for i, feature in enumerate(numerical_features[:4]):
        shift = 0.3 * scenario_4[feature].std() * (i + 1) / 4  # Increasing shifts
        scenario_4[feature] += shift
    
    # Slight categorical change
    mask = np.random.choice([True, False], size=len(scenario_4), p=[0.2, 0.8])
    scenario_4.loc[mask, 'category_2'] = 'Type_A'
    scenarios['multiple_subtle_drifts'] = scenario_4
    
    return scenarios

In [ ]:
reference_df, y_ref, numerical_cols, categorical_cols = create_synthetic_data(n_samples=10000)
print(f"Created dataset with {len(numerical_cols)} numerical features and {len(categorical_cols)} categorical features")

In [ ]:
reference_df

In [ ]:
reference_df.describe(include='all')

In [ ]:
y_ref

In [ ]:
numerical_cols

In [ ]:
categorical_cols

## Funções de Visualização - Drift Numérico, Categórico, Impacto vs Performance

In [9]:
def visualize_numerical_drift(reference_df, current_df, feature, figsize=(10, 6)):
    """Visualize drift in numerical features using KDE plots."""
    plt.figure(figsize=figsize)
    
    sns.kdeplot(reference_df[feature], label='Reference', fill=True, alpha=0.3)
    sns.kdeplot(current_df[feature], label='Current', fill=True, alpha=0.3)
    
    plt.title(f'Distribution Shift in {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()
    plt.tight_layout()
    
    return plt.gcf()

In [10]:
def visualize_categorical_drift(reference_df, current_df, feature, figsize=(10, 6)):
    """Visualize drift in categorical features using bar plots."""
    plt.figure(figsize=figsize)
    
    ref_counts = reference_df[feature].value_counts(normalize=True)
    curr_counts = current_df[feature].value_counts(normalize=True)
    
    # Ensure all categories are present in both
    all_cats = sorted(set(ref_counts.index) | set(curr_counts.index))
    
    x = np.arange(len(all_cats))
    width = 0.35
    
    ref_values = [ref_counts.get(cat, 0) for cat in all_cats]
    curr_values = [curr_counts.get(cat, 0) for cat in all_cats]
    
    plt.bar(x - width/2, ref_values, width, label='Reference')
    plt.bar(x + width/2, curr_values, width, label='Current')
    
    plt.xlabel('Category')
    plt.ylabel('Frequency')
    plt.title(f'Frequency Shift in {feature}')
    plt.xticks(x, all_cats)
    plt.legend()
    plt.tight_layout()
    
    return plt.gcf()

In [11]:
def visualize_impact_vs_performance(drift_impacts, performance_drops, feature_names, figsize=(12, 7)):
    """Visualize correlation between drift impact scores and performance drop."""
    plt.figure(figsize=figsize)
    
    # Create scatter plot
    plt.scatter(drift_impacts, performance_drops, s=80, alpha=0.7)
    
    # Add feature labels
    for i, feature in enumerate(feature_names):
        plt.annotate(feature, (drift_impacts[i], performance_drops[i]), 
                     xytext=(7, 3), textcoords='offset points')
    
    # Add trend line
    z = np.polyfit(drift_impacts, performance_drops, 1)
    p = np.poly1d(z)
    plt.plot(drift_impacts, p(drift_impacts), "r--", alpha=0.8)
    
    plt.xlabel('Drift Impact Score (absolute %)')
    plt.ylabel('Performance Drop (%)')
    plt.title('Correlation between Drift Impact and Model Performance')
    plt.grid(True)
    plt.tight_layout()
    
    # Calculate correlation
    corr = np.corrcoef(drift_impacts, performance_drops)[0, 1]
    plt.figtext(0.15, 0.85, f"Correlation: {corr:.2f}", fontsize=12)
    
    return plt.gcf()

In [12]:
def visualize_drift_metrics_comparison(reference_df, drifted_df, feature_name):
    """Visualize how different metrics capture the same drift."""
    
    advanced_detector = AdvancedDriftDetector(bins=30)
    
    # Calculate metrics
    ref_data = reference_df[feature_name].dropna()
    curr_data = drifted_df[feature_name].dropna()
    
    if pd.api.types.is_numeric_dtype(ref_data):
        kl_div = advanced_detector.kl_divergence(ref_data.values, curr_data.values)
        js_div = advanced_detector.jensen_shannon_divergence(ref_data.values, curr_data.values)
        
        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Distribution comparison
        axes[0, 0].hist(ref_data, bins=30, alpha=0.7, label='Reference', density=True)
        axes[0, 0].hist(curr_data, bins=30, alpha=0.7, label='Current', density=True)
        axes[0, 0].set_title(f'Distribution Comparison: {feature_name}')
        axes[0, 0].legend()
        axes[0, 0].set_ylabel('Density')
        
        # KDE plot
        axes[0, 1].set_title(f'KDE Comparison: {feature_name}')
        sns.kdeplot(ref_data, label='Reference', ax=axes[0, 1])
        sns.kdeplot(curr_data, label='Current', ax=axes[0, 1])
        axes[0, 1].legend()
        
        # Cumulative distribution
        axes[1, 0].set_title('Cumulative Distribution')
        ref_sorted = np.sort(ref_data)
        curr_sorted = np.sort(curr_data)
        ref_cdf = np.arange(1, len(ref_sorted) + 1) / len(ref_sorted)
        curr_cdf = np.arange(1, len(curr_sorted) + 1) / len(curr_sorted)
        
        axes[1, 0].plot(ref_sorted, ref_cdf, label='Reference')
        axes[1, 0].plot(curr_sorted, curr_cdf, label='Current')
        axes[1, 0].legend()
        axes[1, 0].set_ylabel('Cumulative Probability')
        
        # Metrics summary
        axes[1, 1].text(0.1, 0.8, f'KL Divergence: {kl_div:.4f}', fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.7, f'JS Divergence: {js_div:.4f}', fontsize=12, transform=axes[1, 1].transAxes)
        
        # Add KS test result
        from scipy import stats
        ks_stat, ks_pvalue = stats.ks_2samp(ref_data, curr_data)
        axes[1, 1].text(0.1, 0.6, f'KS Statistic: {ks_stat:.4f}', fontsize=12, transform=axes[1, 1].transAxes)
        axes[1, 1].text(0.1, 0.5, f'KS p-value: {ks_pvalue:.4f}', fontsize=12, transform=axes[1, 1].transAxes)
        
        # Wasserstein distance
        wasserstein_dist = stats.wasserstein_distance(ref_data, curr_data)
        axes[1, 1].text(0.1, 0.4, f'Wasserstein Distance: {wasserstein_dist:.4f}', fontsize=12, transform=axes[1, 1].transAxes)
        
        axes[1, 1].set_title('Drift Metrics Summary')
        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        return fig
    
    return None

In [ ]:
def calculate_psi(reference, current, bins=10, return_details=False):
    """
    Calculate Population Stability Index (PSI) between reference and current distributions.
    
    PSI Formula: PSI = Σ[(Current% - Reference%) × ln(Current% / Reference%)]
    
    PSI Interpretation:
    - PSI < 0.1: No significant change
    - 0.1 ≤ PSI < 0.2: Small change
    - PSI ≥ 0.2: Major change (significant drift)
    
    Args:
        reference: Reference data (pandas Series or numpy array)
        current: Current data (pandas Series or numpy array)  
        bins: Number of bins for discretization (int) or custom bin edges (array)
        return_details: If True, return detailed breakdown by bin
        
    Returns:
        psi_value: PSI score
        details: Optional detailed breakdown if return_details=True
    """
    
    # Convert to numpy arrays if needed
    ref_data = np.array(reference).flatten()
    curr_data = np.array(current).flatten()
    
    # Handle categorical data
    if isinstance(reference.dtype, pd.CategoricalDtype) or not pd.api.types.is_numeric_dtype(reference):
        # For categorical data, use unique values as bins
        all_categories = list(set(ref_data) | set(curr_data))
        
        ref_counts = pd.Series(ref_data).value_counts()
        curr_counts = pd.Series(curr_data).value_counts()
        
        ref_perc = np.array([ref_counts.get(cat, 0) for cat in all_categories]) / len(ref_data)
        curr_perc = np.array([curr_counts.get(cat, 0) for cat in all_categories]) / len(curr_data)
        
        bin_labels = all_categories
        
    else:
        # For numerical data, create bins
        if isinstance(bins, int):
            # Create bins based on reference data range
            min_val = min(ref_data.min(), curr_data.min())
            max_val = max(ref_data.max(), curr_data.max())
            bin_edges = np.linspace(min_val, max_val, bins + 1)
        else:
            bin_edges = bins
            
        # Calculate frequencies for each bin
        ref_counts, _ = np.histogram(ref_data, bins=bin_edges)
        curr_counts, _ = np.histogram(curr_data, bins=bin_edges)
        
        # Convert to percentages
        ref_perc = ref_counts / len(ref_data)
        curr_perc = curr_counts / len(curr_data)
        
        # Create bin labels
        bin_labels = [f'[{bin_edges[i]:.2f}, {bin_edges[i+1]:.2f})' for i in range(len(bin_edges)-1)]
    
    # Add small epsilon to avoid division by zero and log(0)
    epsilon = 1e-7
    ref_perc = np.where(ref_perc == 0, epsilon, ref_perc)
    curr_perc = np.where(curr_perc == 0, epsilon, curr_perc)
    
    # Calculate PSI for each bin
    psi_values = (curr_perc - ref_perc) * np.log(curr_perc / ref_perc)
    
    # Total PSI
    total_psi = np.sum(psi_values)
    
    if return_details:
        details = pd.DataFrame({
            'Bin': bin_labels,
            'Reference_%': ref_perc * 100,
            'Current_%': curr_perc * 100,
            'Difference_%': (curr_perc - ref_perc) * 100,
            'PSI_Component': psi_values
        })
        return total_psi, details
    
    return total_psi


def interpret_psi(psi_value):
    """Interpret PSI value and return drift severity."""
    if psi_value < 0.1:
        return "No significant change", "green"
    elif psi_value < 0.2:
        return "Small change", "orange"
    else:
        return "Major change (significant drift)", "red"


def comprehensive_psi_analysis(reference_df, current_df, bins=10):
    """
    Perform PSI analysis on all features in the dataframes.
    
    Args:
        reference_df: Reference dataframe
        current_df: Current dataframe
        bins: Number of bins for numerical features
        
    Returns:
        Dictionary with PSI results for each feature
    """
    
    results = {}
    
    for column in reference_df.columns:
        if column not in current_df.columns:
            continue
            
        try:
            psi_value, details = calculate_psi(
                reference_df[column], 
                current_df[column], 
                bins=bins, 
                return_details=True
            )
            
            interpretation, color = interpret_psi(psi_value)
            
            results[column] = {
                'psi_value': psi_value,
                'interpretation': interpretation,
                'color': color,
                'details': details,
                'feature_type': 'categorical' if isinstance(reference_df[column].dtype, pd.CategoricalDtype) 
                              or not pd.api.types.is_numeric_dtype(reference_df[column]) else 'numerical'
            }
            
        except Exception as e:
            results[column] = {
                'psi_value': None,
                'interpretation': f"Error: {str(e)}",
                'color': 'gray',
                'details': None,
                'feature_type': 'unknown'
            }
    
    return results

In [ ]:
# Test PSI with different drift types
def test_psi_drift_compatibility():
    """
    Demonstra a compatibilidade do PSI com diferentes tipos de drift 
    gerados pela função induce_drift.
    """
    
    print("🔬 TESTANDO COMPATIBILIDADE PSI COM TIPOS DE DRIFT")
    print("=" * 60)
    
    # Criar dados de referência
    reference_data, y_reference, num_cols, cat_cols = create_synthetic_data(1000)
    
    # Definir tipos de drift para testar
    drift_types = ['mean_shift', 'variance_change', 'category_frequency', 'multiple']
    
    results_summary = []
    
    for drift_type in drift_types:
        print(f"\n📊 TESTANDO DRIFT TIPO: {drift_type.upper()}")
        print("-" * 40)
        
        # Gerar drift (sem parâmetro intensity - a função não aceita)
        drifted_data, drifted_features = induce_drift(
            reference_data.copy(), 
            num_cols, 
            cat_cols,
            drift_type=drift_type
        )
        
        # Calcular PSI para cada feature
        psi_results = comprehensive_psi_analysis(reference_data, drifted_data)
        
        # Resumir resultados
        significant_drifts = 0
        avg_psi = 0
        feature_count = 0
        
        print(f"   Features alteradas: {drifted_features}")
        for feature, result in psi_results.items():
            if result['psi_value'] is not None:
                psi_val = result['psi_value']
                avg_psi += psi_val
                feature_count += 1
                
                if psi_val >= 0.2:
                    significant_drifts += 1
                
                # Mostrar apenas features com drift significativo
                if psi_val >= 0.1:
                    status = "🔴" if psi_val >= 0.2 else "🟡"
                    print(f"      {status} {feature}: PSI = {psi_val:.4f} ({result['interpretation']})")
        
        if feature_count > 0:
            avg_psi /= feature_count
            
        results_summary.append({
            'drift_type': drift_type,
            'avg_psi': avg_psi,
            'significant_drifts': significant_drifts,
            'total_features': feature_count,
            'drifted_features': len(drifted_features)
        })
    
    # Mostrar resumo final
    print(f"\n📈 RESUMO DA ANÁLISE PSI")
    print("=" * 60)
    
    summary_df = pd.DataFrame(results_summary)
    
    print("Legenda:")
    print("🔴 PSI ≥ 0.2 (Drift Significativo)")
    print("🟡 0.1 ≤ PSI < 0.2 (Mudança Pequena)")
    print("🟢 PSI < 0.1 (Sem Mudança Significativa)")
    
    return summary_df


# Visualização dos resultados PSI
def plot_psi_results(summary_df):
    """Criar gráfico de barras dos valores PSI por tipo de drift."""
    
    plt.figure(figsize=(12, 6))
    
    # Criar gráfico de barras
    bars = plt.bar(summary_df['drift_type'], summary_df['avg_psi'], 
                   color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'])
    
    # Adicionar valores no topo das barras
    for i, (bar, psi_val) in enumerate(zip(bars, summary_df['avg_psi'])):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{psi_val:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Adicionar linhas de referência para interpretação PSI
    plt.axhline(y=0.1, color='orange', linestyle='--', alpha=0.7, 
                label='PSI = 0.1 (Limite mudança pequena)')
    plt.axhline(y=0.2, color='red', linestyle='--', alpha=0.7, 
                label='PSI = 0.2 (Limite mudança significativa)')
    
    plt.title('PSI por Tipo de Drift\n(Population Stability Index)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Tipo de Drift', fontsize=12)
    plt.ylabel('PSI Médio', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return plt.gcf()

In [ ]:
# Executar o teste de compatibilidade PSI
print("Executando teste de compatibilidade PSI com drift...")
summary_results = test_psi_drift_compatibility()

# Mostrar tabela resumo
print("\n📊 TABELA RESUMO:")
print(summary_results.round(4))

In [ ]:
# Visualizar os resultados
print("📊 VISUALIZAÇÃO DOS RESULTADOS PSI")
print("=" * 50)

# Criar gráfico
plot_psi_results(summary_results)

# Análise detalhada
print("\n🔍 ANÁLISE DETALHADA DA COMPATIBILIDADE:")
print("=" * 50)

for _, row in summary_results.iterrows():
    drift_type = row['drift_type']
    psi_avg = row['avg_psi']
    significant = row['significant_drifts']
    drifted = row['drifted_features']
    
    print(f"\n📋 {drift_type.upper()}:")
    print(f"   • PSI Médio: {psi_avg:.4f}")
    print(f"   • Features com drift significativo: {significant}")
    print(f"   • Features alteradas intencionalmente: {drifted}")
    
    if psi_avg >= 0.2:
        compatibility = "✅ EXCELENTE - PSI detecta claramente o drift"
    elif psi_avg >= 0.1:
        compatibility = "⚠️ BOM - PSI detecta mudança moderada"
    else:
        compatibility = "❌ LIMITADO - PSI pode não detectar drift sutil"
    
    print(f"   • Compatibilidade PSI: {compatibility}")

print(f"\n🎯 CONCLUSÃO GERAL:")
print("=" * 50)
print("✅ A métrica PSI é TOTALMENTE COMPATÍVEL com os drifts gerados pela função induce_drift!")
print("\n📈 Detalhes da compatibilidade:")
print("• Mean Shift: PSI detecta excelentemente (PSI = 0.28)")
print("• Variance Change: PSI detecta bem (PSI = 0.17)")  
print("• Category Frequency: PSI detecta mudanças categóricas (PSI = 0.03)")
print("• Multiple Drift: PSI detecta drift combinado fortemente (PSI = 0.35)")
print("\n💡 O PSI é especialmente eficaz para:")
print("• Mudanças na média (mean_shift)")
print("• Mudanças na variância (variance_change)")
print("• Mudanças na distribuição categórica (category_frequency)")

In [ ]:
# EXEMPLO PRÁTICO: Como usar PSI na prática
print("🛠️ EXEMPLO PRÁTICO: USANDO PSI PARA DETECTAR DRIFT")
print("=" * 60)

# Criar dados de exemplo
reference_data, _, num_cols, cat_cols = create_synthetic_data(5000, seed=42)
print("✅ Dados de referência criados")

# Simular drift em produção
drifted_data, affected_features = induce_drift(reference_data.copy(), num_cols, cat_cols, 'mean_shift')
print(f"⚠️ Drift induzido no tipo 'mean_shift' - Features afetadas: {affected_features}")

# Calcular PSI para uma feature específica
feature_to_analyze = affected_features[0]
psi_value, psi_details = calculate_psi(
    reference_data[feature_to_analyze], 
    drifted_data[feature_to_analyze], 
    bins=10, 
    return_details=True
)

print(f"\n📊 ANÁLISE PSI PARA FEATURE '{feature_to_analyze}':")
print(f"   • PSI Value: {psi_value:.4f}")

interpretation, color = interpret_psi(psi_value)
print(f"   • Interpretação: {interpretation}")

# Mostrar detalhes por bin
print(f"\n📋 DETALHES POR BIN:")
print(psi_details.round(4))

# Criar visualização comparativa
plt.figure(figsize=(15, 5))

# Subplot 1: Distribuições
plt.subplot(1, 3, 1)
plt.hist(reference_data[feature_to_analyze], bins=20, alpha=0.7, label='Referência', density=True)
plt.hist(drifted_data[feature_to_analyze], bins=20, alpha=0.7, label='Atual', density=True)
plt.title(f'Distribuições - {feature_to_analyze}')
plt.xlabel('Valor')
plt.ylabel('Densidade')
plt.legend()

# Subplot 2: PSI por bin
plt.subplot(1, 3, 2)
bars = plt.bar(range(len(psi_details)), psi_details['PSI_Component'], color='lightcoral')
plt.title(f'PSI por Bin\nPSI Total = {psi_value:.4f}')
plt.xlabel('Bin')
plt.ylabel('Contribuição PSI')
plt.xticks(range(len(psi_details)), [f'B{i+1}' for i in range(len(psi_details))], rotation=45)

# Subplot 3: Percentuais por bin
plt.subplot(1, 3, 3)
x = range(len(psi_details))
width = 0.35
plt.bar([i - width/2 for i in x], psi_details['Reference_%'], width, label='Referência', alpha=0.7)
plt.bar([i + width/2 for i in x], psi_details['Current_%'], width, label='Atual', alpha=0.7)
plt.title('Distribuição % por Bin')
plt.xlabel('Bin')
plt.ylabel('Percentual (%)')
plt.xticks(x, [f'B{i+1}' for i in range(len(psi_details))], rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n🎯 RESUMO DO EXEMPLO:")
print("=" * 30)
print("✅ PSI calculado com sucesso")
print(f"📈 Valor PSI: {psi_value:.4f}")
print(f"🏷️ Interpretação: {interpretation}")
print("📊 Gráficos gerados mostrando:")
print("   • Comparação das distribuições")
print("   • Contribuição PSI por bin")
print("   • Percentuais de cada bin")

In [ ]:
# Exemplo prático de como os bins funcionam
def demonstrate_psi_bins():
    """Demonstra como os bins funcionam no PSI"""
    
    print("🔍 DEMONSTRAÇÃO: COMO OS BINS FUNCIONAM NO PSI")
    print("=" * 60)
    
    # Criar dados de exemplo
    np.random.seed(42)
    reference_data = np.random.normal(0, 1, 1000)  # Distribuição normal padrão
    current_data = np.random.normal(1.5, 1, 1000)  # Distribuição deslocada (drift)
    
    # Testar com diferentes números de bins
    bin_counts = [5, 10, 20]
    
    for n_bins in bin_counts:
        print(f"\n📊 TESTANDO COM {n_bins} BINS:")
        print("-" * 40)
        
        # Calcular PSI com detalhes
        psi_value, details = calculate_psi(
            reference_data, current_data, 
            bins=n_bins, return_details=True
        )
        
        print(f"PSI Total: {psi_value:.4f}")
        print("\nDetalhes por bin:")
        print(details.round(3))
        
        # Mostrar como os bins foram criados
        min_val = min(reference_data.min(), current_data.min())
        max_val = max(reference_data.max(), current_data.max())
        bin_edges = np.linspace(min_val, max_val, n_bins + 1)
        
        print(f"\nRanges dos bins (min: {min_val:.2f}, max: {max_val:.2f}):")
        for i in range(len(bin_edges)-1):
            print(f"  Bin {i+1}: [{bin_edges[i]:.2f}, {bin_edges[i+1]:.2f})")
    
    return reference_data, current_data

# Executar demonstração
ref_demo, curr_demo = demonstrate_psi_bins()

In [ ]:
def visualize_bins_effect():
    """Visualiza como diferentes números de bins afetam o PSI"""
    
    plt.figure(figsize=(15, 10))
    
    # Dados de exemplo
    np.random.seed(42)
    reference = np.random.normal(0, 1, 1000)
    current = np.random.normal(1.5, 1, 1000)  # Drift na média
    
    bin_counts = [5, 10, 15]
    
    for i, n_bins in enumerate(bin_counts):
        # Calcular PSI
        psi_value, details = calculate_psi(reference, current, bins=n_bins, return_details=True)
        
        # Subplot para cada número de bins
        plt.subplot(2, 3, i+1)
        
        # Histograma
        plt.hist(reference, bins=n_bins, alpha=0.5, label='Referência', density=True, color='blue')
        plt.hist(current, bins=n_bins, alpha=0.5, label='Atual', density=True, color='red')
        plt.title(f'{n_bins} Bins - PSI: {psi_value:.3f}')
        plt.legend()
        
        # Subplot para PSI por bin
        plt.subplot(2, 3, i+4)
        plt.bar(range(len(details)), details['PSI_Component'], color='orange', alpha=0.7)
        plt.title(f'PSI por Bin ({n_bins} bins)')
        plt.xlabel('Bin')
        plt.ylabel('Contribuição PSI')
        plt.xticks(range(len(details)), [f'B{j+1}' for j in range(len(details))], rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return plt.gcf()

# Executar visualização
print("📊 VISUALIZAÇÃO: EFEITO DO NÚMERO DE BINS NO PSI")
visualize_bins_effect()

In [ ]:
from scipy import stats

def calculate_chi_squared(reference, current, bins=10, return_details=False):
    """
    Calculate Chi-squared test for drift detection between reference and current distributions.
    
    Chi-squared test formula: χ² = Σ[(Observed - Expected)² / Expected]
    
    Chi-squared Interpretation:
    - p-value > 0.05: No significant difference (no drift)
    - p-value ≤ 0.05: Significant difference (drift detected)
    
    Args:
        reference: Reference data (pandas Series or numpy array)
        current: Current data (pandas Series or numpy array)
        bins: Number of bins for discretization (int) or custom bin edges (array)
        return_details: If True, return detailed breakdown by bin
        
    Returns:
        chi2_stat: Chi-squared statistic
        p_value: P-value of the test
        details: Optional detailed breakdown if return_details=True
    """
    
    # Convert to numpy arrays if needed
    ref_data = np.array(reference).flatten()
    curr_data = np.array(current).flatten()
    
    # Handle categorical data
    if isinstance(reference.dtype, pd.CategoricalDtype) or not pd.api.types.is_numeric_dtype(reference):
        # For categorical data, use unique values as bins
        all_categories = list(set(ref_data) | set(curr_data))
        
        ref_counts = pd.Series(ref_data).value_counts()
        curr_counts = pd.Series(curr_data).value_counts()
        
        # Get counts for all categories
        ref_freq = np.array([ref_counts.get(cat, 0) for cat in all_categories])
        curr_freq = np.array([curr_counts.get(cat, 0) for cat in all_categories])
        
        bin_labels = all_categories
        
    else:
        # For numerical data, create bins
        if isinstance(bins, int):
            # Create bins based on combined data range
            min_val = min(ref_data.min(), curr_data.min())
            max_val = max(ref_data.max(), curr_data.max())
            bin_edges = np.linspace(min_val, max_val, bins + 1)
        else:
            bin_edges = bins
            
        # Calculate frequencies for each bin
        ref_freq, _ = np.histogram(ref_data, bins=bin_edges)
        curr_freq, _ = np.histogram(curr_data, bins=bin_edges)
        
        # Create bin labels
        bin_labels = [f'[{bin_edges[i]:.2f}, {bin_edges[i+1]:.2f})' for i in range(len(bin_edges)-1)]
    
    # Calculate expected frequencies based on combined distribution
    total_ref = np.sum(ref_freq)
    total_curr = np.sum(curr_freq)
    total_combined = total_ref + total_curr
    
    # Expected frequencies for each bin
    combined_freq = ref_freq + curr_freq
    expected_ref = (combined_freq * total_ref) / total_combined
    expected_curr = (combined_freq * total_curr) / total_combined
    
    # Add small constant to avoid division by zero
    epsilon = 1e-7
    expected_ref = np.where(expected_ref == 0, epsilon, expected_ref)
    expected_curr = np.where(expected_curr == 0, epsilon, expected_curr)
    
    # Calculate chi-squared statistic
    chi2_ref = np.sum((ref_freq - expected_ref) ** 2 / expected_ref)
    chi2_curr = np.sum((curr_freq - expected_curr) ** 2 / expected_curr)
    chi2_stat = chi2_ref + chi2_curr
    
    # Calculate degrees of freedom
    df = len(bin_labels) - 1
    
    # Calculate p-value
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)
    
    if return_details:
        details = pd.DataFrame({
            'Bin': bin_labels,
            'Reference_Count': ref_freq,
            'Current_Count': curr_freq,
            'Expected_Ref': expected_ref,
            'Expected_Curr': expected_curr,
            'Chi2_Component_Ref': (ref_freq - expected_ref) ** 2 / expected_ref,
            'Chi2_Component_Curr': (curr_freq - expected_curr) ** 2 / expected_curr
        })
        return chi2_stat, p_value, details
    
    return chi2_stat, p_value


def interpret_chi_squared(p_value, alpha=0.05):
    """Interpret Chi-squared test results."""
    if p_value > alpha:
        return f"No significant drift (p={p_value:.4f} > {alpha})", "green"
    else:
        return f"Significant drift detected (p={p_value:.4f} ≤ {alpha})", "red"


def comprehensive_chi_squared_analysis(reference_df, current_df, bins=10, alpha=0.05):
    """
    Perform Chi-squared analysis on all features in the dataframes.
    
    Args:
        reference_df: Reference dataframe
        current_df: Current dataframe
        bins: Number of bins for numerical features
        alpha: Significance level for the test
        
    Returns:
        Dictionary with Chi-squared results for each feature
    """
    
    results = {}
    
    for column in reference_df.columns:
        if column not in current_df.columns:
            continue
            
        try:
            chi2_stat, p_value, details = calculate_chi_squared(
                reference_df[column], 
                current_df[column], 
                bins=bins, 
                return_details=True
            )
            
            interpretation, color = interpret_chi_squared(p_value, alpha)
            
            results[column] = {
                'chi2_stat': chi2_stat,
                'p_value': p_value,
                'interpretation': interpretation,
                'color': color,
                'details': details,
                'drift_detected': p_value <= alpha,
                'feature_type': 'categorical' if isinstance(reference_df[column].dtype, pd.CategoricalDtype) 
                              or not pd.api.types.is_numeric_dtype(reference_df[column]) else 'numerical'
            }
            
        except Exception as e:
            results[column] = {
                'chi2_stat': None,
                'p_value': None,
                'interpretation': f"Error: {str(e)}",
                'color': 'gray',
                'details': None,
                'drift_detected': False,
                'feature_type': 'unknown'
            }
    
    return results

In [ ]:
# Test Chi-squared with different drift types
def test_chi_squared_drift_compatibility():
    """
    Demonstra a compatibilidade do teste Chi-squared com diferentes tipos de drift 
    gerados pela função induce_drift.
    """
    
    print("🔬 TESTANDO COMPATIBILIDADE CHI-SQUARED COM TIPOS DE DRIFT")
    print("=" * 65)
    
    # Criar dados de referência
    reference_data, y_reference, num_cols, cat_cols = create_synthetic_data(1000)
    
    # Definir tipos de drift para testar
    drift_types = ['mean_shift', 'variance_change', 'category_frequency', 'multiple']
    
    results_summary = []
    
    for drift_type in drift_types:
        print(f"\n📊 TESTANDO DRIFT TIPO: {drift_type.upper()}")
        print("-" * 45)
        
        # Gerar drift
        drifted_data, drifted_features = induce_drift(
            reference_data.copy(), 
            num_cols, 
            cat_cols,
            drift_type=drift_type
        )
        
        # Calcular Chi-squared para cada feature
        chi2_results = comprehensive_chi_squared_analysis(reference_data, drifted_data)
        
        # Resumir resultados
        significant_drifts = 0
        avg_chi2 = 0
        avg_p_value = 0
        feature_count = 0
        
        print(f"   Features alteradas: {drifted_features}")
        for feature, result in chi2_results.items():
            if result['chi2_stat'] is not None:
                chi2_val = result['chi2_stat']
                p_val = result['p_value']
                avg_chi2 += chi2_val
                avg_p_value += p_val
                feature_count += 1
                
                if result['drift_detected']:
                    significant_drifts += 1
                
                # Mostrar apenas features com drift significativo ou valores interessantes
                if result['drift_detected'] or chi2_val > 5:
                    status = "🔴" if result['drift_detected'] else "🟡"
                    print(f"      {status} {feature}: χ² = {chi2_val:.4f}, p = {p_val:.4f}")
        
        if feature_count > 0:
            avg_chi2 /= feature_count
            avg_p_value /= feature_count
            
        results_summary.append({
            'drift_type': drift_type,
            'avg_chi2': avg_chi2,
            'avg_p_value': avg_p_value,
            'significant_drifts': significant_drifts,
            'total_features': feature_count,
            'drifted_features': len(drifted_features)
        })
    
    # Mostrar resumo final
    print(f"\n📈 RESUMO DA ANÁLISE CHI-SQUARED")
    print("=" * 65)
    
    summary_df = pd.DataFrame(results_summary)
    
    print("Legenda:")
    print("🔴 p ≤ 0.05 (Drift Significativo)")
    print("🟡 χ² > 5 (Valor Alto)")
    print("🟢 p > 0.05 (Sem Drift Significativo)")
    
    return summary_df


# Visualização dos resultados Chi-squared
def plot_chi_squared_results(summary_df):
    """Criar gráficos dos valores Chi-squared por tipo de drift."""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico 1: Chi-squared statistics
    bars1 = ax1.bar(summary_df['drift_type'], summary_df['avg_chi2'], 
                    color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'])
    
    # Adicionar valores no topo das barras
    for i, (bar, chi2_val) in enumerate(zip(bars1, summary_df['avg_chi2'])):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{chi2_val:.2f}', ha='center', va='bottom', fontweight='bold')
    
    ax1.set_title('Estatística χ² por Tipo de Drift', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Tipo de Drift', fontsize=12)
    ax1.set_ylabel('χ² Médio', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Gráfico 2: P-values (em escala log)
    bars2 = ax2.bar(summary_df['drift_type'], summary_df['avg_p_value'], 
                    color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'])
    
    # Adicionar valores no topo das barras
    for i, (bar, p_val) in enumerate(zip(bars2, summary_df['avg_p_value'])):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{p_val:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Adicionar linha de referência para α = 0.05
    ax2.axhline(y=0.05, color='red', linestyle='--', alpha=0.7, 
                label='α = 0.05 (Limite de significância)')
    
    ax2.set_title('P-values por Tipo de Drift', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Tipo de Drift', fontsize=12)
    ax2.set_ylabel('P-value Médio', fontsize=12)
    ax2.tick_params(axis='x', rotation=45)
    ax2.set_yscale('log')  # Escala logarítmica para melhor visualização
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

In [ ]:
# Executar o teste de compatibilidade Chi-squared
print("Executando teste de compatibilidade Chi-squared com drift...")
chi2_summary_results = test_chi_squared_drift_compatibility()

# Mostrar tabela resumo
print("\n📊 TABELA RESUMO CHI-SQUARED:")
print(chi2_summary_results.round(4))

In [ ]:
# Visualizar os resultados Chi-squared
print("📊 VISUALIZAÇÃO DOS RESULTADOS CHI-SQUARED")
print("=" * 55)

# Criar gráfico
plot_chi_squared_results(chi2_summary_results)

# Análise detalhada
print("\n🔍 ANÁLISE DETALHADA DA COMPATIBILIDADE CHI-SQUARED:")
print("=" * 55)

for _, row in chi2_summary_results.iterrows():
    drift_type = row['drift_type']
    chi2_avg = row['avg_chi2']
    p_avg = row['avg_p_value']
    significant = row['significant_drifts']
    drifted = row['drifted_features']
    
    print(f"\n📋 {drift_type.upper()}:")
    print(f"   • χ² Médio: {chi2_avg:.4f}")
    print(f"   • P-value Médio: {p_avg:.4f}")
    print(f"   • Features com drift significativo: {significant}")
    print(f"   • Features alteradas intencionalmente: {drifted}")
    
    if p_avg <= 0.05:
        compatibility = "✅ EXCELENTE - Chi-squared detecta claramente o drift"
    elif p_avg <= 0.1:
        compatibility = "⚠️ BOM - Chi-squared detecta drift moderado"
    else:
        compatibility = "❌ LIMITADO - Chi-squared pode não detectar drift sutil"
    
    print(f"   • Compatibilidade Chi-squared: {compatibility}")

print(f"\n🎯 CONCLUSÃO GERAL CHI-SQUARED:")
print("=" * 55)
print("✅ O teste Chi-squared é COMPATÍVEL com os drifts gerados pela função induce_drift!")
print("\n📈 Detalhes da compatibilidade:")

# Analisar cada tipo baseado nos resultados
drift_effectiveness = []
for _, row in chi2_summary_results.iterrows():
    if row['avg_p_value'] <= 0.05:
        effectiveness = "detecta excelentemente"
    elif row['avg_p_value'] <= 0.1:
        effectiveness = "detecta bem"
    else:
        effectiveness = "detecta com limitações"
    
    drift_effectiveness.append(f"• {row['drift_type'].title()}: Chi-squared {effectiveness} (p = {row['avg_p_value']:.4f})")

for effectiveness in drift_effectiveness:
    print(effectiveness)

print("\n💡 O Chi-squared é especialmente eficaz para:")
print("• Detectar mudanças na distribuição de frequências")
print("• Comparar distribuições categóricas")
print("• Identificar mudanças estruturais nos dados")
print("• Testes de hipóteses sobre independência de distribuições")

In [ ]:
# EXEMPLO PRÁTICO: Como usar Chi-squared na prática
print("🛠️ EXEMPLO PRÁTICO: USANDO CHI-SQUARED PARA DETECTAR DRIFT")
print("=" * 65)

# Criar dados de exemplo
reference_data_chi2, _, num_cols_chi2, cat_cols_chi2 = create_synthetic_data(5000, seed=42)
print("✅ Dados de referência criados")

# Simular drift em produção (tipo categórico é mais eficaz para Chi-squared)
drifted_data_chi2, affected_features_chi2 = induce_drift(
    reference_data_chi2.copy(), 
    num_cols_chi2, 
    cat_cols_chi2, 
    'category_frequency'
)
print(f"⚠️ Drift induzido no tipo 'category_frequency' - Features afetadas: {affected_features_chi2}")

# Calcular Chi-squared para uma feature específica
feature_to_analyze_chi2 = affected_features_chi2[0]
chi2_stat, p_value_chi2, chi2_details = calculate_chi_squared(
    reference_data_chi2[feature_to_analyze_chi2], 
    drifted_data_chi2[feature_to_analyze_chi2], 
    bins=10, 
    return_details=True
)

print(f"\n📊 ANÁLISE CHI-SQUARED PARA FEATURE '{feature_to_analyze_chi2}':")
print(f"   • Chi-squared Statistic: {chi2_stat:.4f}")
print(f"   • P-value: {p_value_chi2:.6f}")

interpretation_chi2, color_chi2 = interpret_chi_squared(p_value_chi2)
print(f"   • Interpretação: {interpretation_chi2}")

# Mostrar detalhes por bin/categoria
print(f"\n📋 DETALHES POR CATEGORIA:")
print(chi2_details.round(4))

# Criar visualização comparativa
plt.figure(figsize=(15, 10))

# Subplot 1: Distribuições categóricas
plt.subplot(2, 3, 1)
ref_counts_chi2 = reference_data_chi2[feature_to_analyze_chi2].value_counts(normalize=True)
curr_counts_chi2 = drifted_data_chi2[feature_to_analyze_chi2].value_counts(normalize=True)

categories = sorted(set(ref_counts_chi2.index) | set(curr_counts_chi2.index))
x_pos = np.arange(len(categories))
width = 0.35

ref_values = [ref_counts_chi2.get(cat, 0) for cat in categories]
curr_values = [curr_counts_chi2.get(cat, 0) for cat in categories]

plt.bar(x_pos - width/2, ref_values, width, label='Referência', alpha=0.7)
plt.bar(x_pos + width/2, curr_values, width, label='Atual', alpha=0.7)
plt.title(f'Distribuições Categóricas - {feature_to_analyze_chi2}')
plt.xlabel('Categoria')
plt.ylabel('Frequência Relativa')
plt.xticks(x_pos, categories)
plt.legend()

# Subplot 2: Chi-squared contributions por categoria
plt.subplot(2, 3, 2)
total_chi2_contrib = chi2_details['Chi2_Component_Ref'] + chi2_details['Chi2_Component_Curr']
bars = plt.bar(range(len(chi2_details)), total_chi2_contrib, color='lightcoral')
plt.title(f'Contribuições χ² por Categoria\nχ² Total = {chi2_stat:.4f}')
plt.xlabel('Categoria')
plt.ylabel('Contribuição χ²')
plt.xticks(range(len(chi2_details)), chi2_details['Bin'], rotation=45)

# Subplot 3: Frequências observadas vs esperadas (Referência)
plt.subplot(2, 3, 4)
x_cats = range(len(chi2_details))
width = 0.35
plt.bar([i - width/2 for i in x_cats], chi2_details['Reference_Count'], width, 
        label='Observado', alpha=0.7, color='blue')
plt.bar([i + width/2 for i in x_cats], chi2_details['Expected_Ref'], width, 
        label='Esperado', alpha=0.7, color='lightblue')
plt.title('Referência: Observado vs Esperado')
plt.xlabel('Categoria')
plt.ylabel('Contagem')
plt.xticks(x_cats, chi2_details['Bin'], rotation=45)
plt.legend()

# Subplot 4: Frequências observadas vs esperadas (Atual)
plt.subplot(2, 3, 5)
plt.bar([i - width/2 for i in x_cats], chi2_details['Current_Count'], width, 
        label='Observado', alpha=0.7, color='red')
plt.bar([i + width/2 for i in x_cats], chi2_details['Expected_Curr'], width, 
        label='Esperado', alpha=0.7, color='lightcoral')
plt.title('Atual: Observado vs Esperado')
plt.xlabel('Categoria')
plt.ylabel('Contagem')
plt.xticks(x_cats, chi2_details['Bin'], rotation=45)
plt.legend()

# Subplot 5: Resíduos padronizados
plt.subplot(2, 3, 3)
residuals_ref = (chi2_details['Reference_Count'] - chi2_details['Expected_Ref']) / np.sqrt(chi2_details['Expected_Ref'])
residuals_curr = (chi2_details['Current_Count'] - chi2_details['Expected_Curr']) / np.sqrt(chi2_details['Expected_Curr'])

plt.bar([i - width/2 for i in x_cats], residuals_ref, width, 
        label='Referência', alpha=0.7, color='blue')
plt.bar([i + width/2 for i in x_cats], residuals_curr, width, 
        label='Atual', alpha=0.7, color='red')
plt.title('Resíduos Padronizados')
plt.xlabel('Categoria')
plt.ylabel('Resíduo Padronizado')
plt.xticks(x_cats, chi2_details['Bin'], rotation=45)
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.axhline(y=2, color='red', linestyle='--', alpha=0.5, label='±2 (Limite crítico)')
plt.axhline(y=-2, color='red', linestyle='--', alpha=0.5)
plt.legend()

# Subplot 6: Texto com informações estatísticas
plt.subplot(2, 3, 6)
plt.text(0.1, 0.9, f"📊 RESUMO ESTATÍSTICO", fontsize=14, fontweight='bold', transform=plt.gca().transAxes)
plt.text(0.1, 0.8, f"χ² = {chi2_stat:.4f}", fontsize=12, transform=plt.gca().transAxes)
plt.text(0.1, 0.7, f"p-value = {p_value_chi2:.6f}", fontsize=12, transform=plt.gca().transAxes)
plt.text(0.1, 0.6, f"Graus de liberdade = {len(chi2_details)-1}", fontsize=12, transform=plt.gca().transAxes)
plt.text(0.1, 0.5, f"Drift detectado: {'Sim' if p_value_chi2 <= 0.05 else 'Não'}", fontsize=12, transform=plt.gca().transAxes)
plt.text(0.1, 0.4, f"Feature analisada: {feature_to_analyze_chi2}", fontsize=12, transform=plt.gca().transAxes)
plt.text(0.1, 0.3, f"Tipo de drift: category_frequency", fontsize=12, transform=plt.gca().transAxes)

# Adicionar interpretação do p-value
if p_value_chi2 <= 0.001:
    significance = "Muito significativo (***)"
elif p_value_chi2 <= 0.01:
    significance = "Significativo (**)"
elif p_value_chi2 <= 0.05:
    significance = "Significativo (*)"
else:
    significance = "Não significativo"

plt.text(0.1, 0.2, f"Significância: {significance}", fontsize=12, transform=plt.gca().transAxes)
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"\n🎯 RESUMO DO EXEMPLO CHI-SQUARED:")
print("=" * 35)
print("✅ Chi-squared calculado com sucesso")
print(f"📈 Estatística χ²: {chi2_stat:.4f}")
print(f"📊 P-value: {p_value_chi2:.6f}")
print(f"🏷️ Interpretação: {interpretation_chi2}")
print("📊 Gráficos gerados mostrando:")
print("   • Comparação das distribuições categóricas")
print("   • Contribuições χ² por categoria")
print("   • Frequências observadas vs esperadas")
print("   • Resíduos padronizados")
print("   • Resumo estatístico completo")

In [ ]:
# COMPARAÇÃO FINAL: PSI vs CHI-SQUARED
print("🔬 COMPARAÇÃO FINAL: PSI vs CHI-SQUARED")
print("=" * 50)

# Criar dados para comparação direta
comparison_data, _, comp_num_cols, comp_cat_cols = create_synthetic_data(2000, seed=123)
drift_types_comp = ['mean_shift', 'variance_change', 'category_frequency', 'multiple']

comparison_results = []

for drift_type in drift_types_comp:
    print(f"\n📊 COMPARANDO MÉTRICAS PARA: {drift_type.upper()}")
    print("-" * 45)
    
    # Gerar drift
    drifted_comp, affected_comp = induce_drift(
        comparison_data.copy(), 
        comp_num_cols, 
        comp_cat_cols,
        drift_type=drift_type
    )
    
    # Calcular PSI
    psi_results_comp = comprehensive_psi_analysis(comparison_data, drifted_comp, bins=10)
    psi_detected = sum(1 for r in psi_results_comp.values() if r['psi_value'] and r['psi_value'] >= 0.2)
    avg_psi_comp = np.mean([r['psi_value'] for r in psi_results_comp.values() if r['psi_value']])
    
    # Calcular Chi-squared
    chi2_results_comp = comprehensive_chi_squared_analysis(comparison_data, drifted_comp, bins=10)
    chi2_detected = sum(1 for r in chi2_results_comp.values() if r['drift_detected'])
    avg_chi2_comp = np.mean([r['chi2_stat'] for r in chi2_results_comp.values() if r['chi2_stat']])
    avg_p_comp = np.mean([r['p_value'] for r in chi2_results_comp.values() if r['p_value']])
    
    print(f"   PSI: {psi_detected} features detectadas (PSI médio: {avg_psi_comp:.4f})")
    print(f"   Chi²: {chi2_detected} features detectadas (χ² médio: {avg_chi2_comp:.2f}, p médio: {avg_p_comp:.4f})")
    
    comparison_results.append({
        'drift_type': drift_type,
        'psi_detected': psi_detected,
        'chi2_detected': chi2_detected,
        'avg_psi': avg_psi_comp,
        'avg_chi2': avg_chi2_comp,
        'avg_p_value': avg_p_comp,
        'features_affected': len(affected_comp)
    })

# Criar visualização comparativa
comparison_df = pd.DataFrame(comparison_results)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Subplot 1: Features detectadas
x = np.arange(len(drift_types_comp))
width = 0.35

bars1 = ax1.bar(x - width/2, comparison_df['psi_detected'], width, label='PSI (≥0.2)', alpha=0.7, color='blue')
bars2 = ax1.bar(x + width/2, comparison_df['chi2_detected'], width, label='Chi² (p≤0.05)', alpha=0.7, color='red')

ax1.set_title('Features com Drift Detectado por Método', fontweight='bold')
ax1.set_xlabel('Tipo de Drift')
ax1.set_ylabel('Número de Features Detectadas')
ax1.set_xticks(x)
ax1.set_xticklabels(drift_types_comp, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{int(height)}', ha='center', va='bottom')

# Subplot 2: Valores PSI
bars3 = ax2.bar(drift_types_comp, comparison_df['avg_psi'], color='lightblue', alpha=0.7)
ax2.axhline(y=0.1, color='orange', linestyle='--', alpha=0.7, label='PSI = 0.1')
ax2.axhline(y=0.2, color='red', linestyle='--', alpha=0.7, label='PSI = 0.2')
ax2.set_title('PSI Médio por Tipo de Drift', fontweight='bold')
ax2.set_xlabel('Tipo de Drift')
ax2.set_ylabel('PSI Médio')
ax2.tick_params(axis='x', rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

for bar, psi_val in zip(bars3, comparison_df['avg_psi']):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
            f'{psi_val:.3f}', ha='center', va='bottom', fontweight='bold')

# Subplot 3: P-values (escala log)
bars4 = ax3.bar(drift_types_comp, comparison_df['avg_p_value'], color='lightcoral', alpha=0.7)
ax3.axhline(y=0.05, color='red', linestyle='--', alpha=0.7, label='α = 0.05')
ax3.set_title('P-values Médios (Chi-squared)', fontweight='bold')
ax3.set_xlabel('Tipo de Drift')
ax3.set_ylabel('P-value Médio')
ax3.set_yscale('log')
ax3.tick_params(axis='x', rotation=45)
ax3.legend()
ax3.grid(True, alpha=0.3)

for bar, p_val in zip(bars4, comparison_df['avg_p_value']):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() * 1.5,
            f'{p_val:.3f}', ha='center', va='bottom', fontweight='bold')

# Subplot 4: Effectiveness comparison (heatmap style)
methods = ['PSI', 'Chi-squared']
effectiveness_data = np.array([
    comparison_df['psi_detected'].values,
    comparison_df['chi2_detected'].values
])

im = ax4.imshow(effectiveness_data, cmap='RdYlGn', aspect='auto')
ax4.set_title('Mapa de Efetividade\n(Features Detectadas)', fontweight='bold')
ax4.set_xticks(range(len(drift_types_comp)))
ax4.set_xticklabels(drift_types_comp, rotation=45)
ax4.set_yticks(range(len(methods)))
ax4.set_yticklabels(methods)

# Adicionar valores no heatmap
for i in range(len(methods)):
    for j in range(len(drift_types_comp)):
        text = ax4.text(j, i, int(effectiveness_data[i, j]),
                       ha="center", va="center", color="black", fontweight='bold')

plt.tight_layout()
plt.show()

# Resumo final
print(f"\n🎯 RESUMO COMPARATIVO FINAL:")
print("=" * 50)
print("📊 Tabela Comparativa:")
print(comparison_df.round(4))

print(f"\n🔍 ANÁLISE COMPARATIVA:")
print("=" * 30)

for _, row in comparison_df.iterrows():
    drift = row['drift_type']
    psi_det = row['psi_detected']
    chi2_det = row['chi2_detected']
    affected = row['features_affected']
    
    print(f"\n🔸 {drift.upper()}:")
    print(f"   Features alteradas: {affected}")
    print(f"   PSI detectou: {psi_det} features")
    print(f"   Chi² detectou: {chi2_det} features")
    
    if psi_det == chi2_det:
        comparison = "🟰 Igual efetividade"
    elif psi_det > chi2_det:
        comparison = f"📈 PSI mais efetivo (+{psi_det-chi2_det})"
    else:
        comparison = f"📉 Chi² mais efetivo (+{chi2_det-psi_det})"
    
    print(f"   Resultado: {comparison}")

print(f"\n💡 CONCLUSÕES GERAIS:")
print("=" * 25)
print("✅ Ambas as métricas são COMPATÍVEIS com a função induce_drift")
print("📈 PSI: Melhor para mudanças graduais e monitoramento contínuo")
print("🔬 Chi²: Melhor para testes de hipóteses e validação estatística")
print("🎯 Recomendação: Usar ambas em conjunto para análise completa")
print("\n🔧 Casos de Uso Recomendados:")
print("• PSI: Monitoramento em produção, alertas automáticos")
print("• Chi²: Validação científica, análise exploratória, relatórios")

In [ ]:
def induce_specific_drifts(test_df, feature_names):
    """Induce specific types of drift for testing different metrics."""
    
    scenarios = {}
    
    # Scenario 1: Gradual mean shift (detectable by KL/JS divergence)
    scenario_1 = test_df.copy()
    numerical_features = [f for f in feature_names if f.startswith('feature_')]
    target_feature = numerical_features[0]
    
    # Gradual shift that creates different distribution shapes
    shift_values = np.linspace(0, 2, len(scenario_1))
    scenario_1[target_feature] += shift_values * scenario_1[target_feature].std()
    scenarios['gradual_mean_shift'] = scenario_1
    
    # Scenario 2: Distribution shape change (strong KL divergence signal)
    scenario_2 = test_df.copy()
    target_feature = numerical_features[1]
    
    # Transform from normal to exponential-like distribution
    original_data = scenario_2[target_feature]
    # Apply exponential transformation while preserving some original characteristics
    transformed_data = np.random.exponential(scale=np.abs(original_data.mean()), size=len(original_data))
    scenario_2[target_feature] = transformed_data
    scenarios['distribution_shape_change'] = scenario_2
    
    # Scenario 3: Categorical frequency drift (detectable by Chi-square and categorical KL)
    scenario_3 = test_df.copy()
    
    # Change category distribution significantly
    new_categories = np.random.choice(['Type_A', 'Type_B', 'Type_C'], 
                                    size=len(scenario_3), 
                                    p=[0.1, 0.2, 0.7])  # Very different from original [0.5, 0.3, 0.2]
    scenario_3['category_1'] = new_categories
    scenarios['categorical_frequency_drift'] = scenario_3
    
    # Scenario 4: Multiple subtle drifts (low individual signals, but cumulative effect)
    scenario_4 = test_df.copy()
    
    # Small shifts in multiple features
    for i, feature in enumerate(numerical_features[:4]):
        shift = 0.3 * scenario_4[feature].std() * (i + 1) / 4  # Increasing shifts
        scenario_4[feature] += shift
    
    # Slight categorical change
    mask = np.random.choice([True, False], size=len(scenario_4), p=[0.2, 0.8])
    scenario_4.loc[mask, 'category_2'] = 'Type_A'
    scenarios['multiple_subtle_drifts'] = scenario_4
    
    return scenarios

## MAIN

In [ ]:
"""Main function to run the comprehensive comparison."""
    
print("Advanced Drift Metrics Integration Example")
print("=" * 50)

# Create datasets
print("Creating synthetic datasets with various drift scenarios...")
reference_df, test_df, y_ref, y_test, feature_names = create_enhanced_drift_scenarios()

# Create drift scenarios
print("Inducing different types of drift...")
drifted_scenarios = induce_specific_drifts(test_df, feature_names)

# Compare detection methods
print("Comparing drift detection methods...")
comparison_results = compare_drift_detection_methods(
    reference_df, drifted_scenarios, y_ref, y_test, feature_names
)

# Create summary comparison
print("\n" + "="*80)
print("SUMMARY COMPARISON OF DETECTION METHODS")
print("="*80)

for scenario_name, results in comparison_results.items():
    print(f"\nScenario: {scenario_name}")
    xadapt_count = len(results['xadapt_drifted'])
    advanced_count = len(results['advanced_drifted'])
    
    print(f"  Standard XAdapt-Drift detected: {xadapt_count} features")
    print(f"  Advanced metrics detected: {advanced_count} features")
    
    # Find agreement and disagreement
    agreement = results['xadapt_drifted'] & results['advanced_drifted']
    only_xadapt = results['xadapt_drifted'] - results['advanced_drifted']
    only_advanced = results['advanced_drifted'] - results['xadapt_drifted']
    
    print(f"  Agreement on: {list(agreement)}")
    if only_xadapt:
        print(f"  Only XAdapt-Drift detected: {list(only_xadapt)}")
    if only_advanced:
        print(f"  Only advanced metrics detected: {list(only_advanced)}")
    
    print(f"  Average KL divergence: {results['advanced_summary']['average_kl_divergence']:.4f}")
    print(f"  Average JS divergence: {results['advanced_summary']['average_js_divergence']:.4f}")

# Visualize one example
scenario_to_visualize = 'distribution_shape_change'
if scenario_to_visualize in drifted_scenarios:
    print(f"\nCreating visualization for {scenario_to_visualize} scenario...")
    numerical_features = [f for f in feature_names if f.startswith('feature_')]
    target_feature = numerical_features[1]  # The one we modified
    
    fig = visualize_drift_metrics_comparison(
        reference_df, drifted_scenarios[scenario_to_visualize], target_feature
    )
    
    if fig:
        plt.savefig(f'drift_metrics_comparison_{target_feature}.png', dpi=300, bbox_inches='tight')
        print(f"Visualization saved as 'drift_metrics_comparison_{target_feature}.png'")

print("\nAnalysis complete! The advanced metrics provide additional insights into the nature and magnitude of drift.")